In [1]:
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch
from calendar import monthrange, month_name
from collections import defaultdict, OrderedDict
from dateutil.relativedelta import relativedelta
from pprint import pprint

import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# Elasticsearch should be running at this URL, otherwise the next command will give and error
ES_URL = "http://localhost:9200" 

es = Elasticsearch(ES_URL)

# URL of the repository to be analysed
org_name = "Cloud-CV"
repo_name = "EvalAI"

# Names of the indices by which the repository will be stored
enriched_index_name = "cloud_git"
raw_index_name = "cloud_git_raw"

# This token will allow us to call GitHub's api without being rate limited 
github_access_token = "af8040bba07cdbae8a6ac420b12a1afae1625c06" # Enter your own token

In [12]:
subprocess.run(['p2o.py', '--enrich', '--index', raw_index_name,
      '--index-enrich', enriched_index_name, '-e', ES_URL,
      '--no_inc', '--debug', 'git',org_name , repo_name,'-t',
                github_access_token, '--sleep-for-rate'])
print("Data retrieved and Index enriched")

Data retrieved and Index enriched


In [10]:
def retrieve_records(index=None):
    "Queries the elasticsearch instance and returns all the documents in the index."
    
    search_result = es.search(index=index, body={"query":{"match_all":{}}})
    total = search_result["hits"]["total"]
    query = {
                "size":total, 
                "query":
                        {
                            "match_all":{}
                        }
            }
    query_result = es.search(index=index, body=query)
    results = [query_result["hits"]["hits"][i]["_source"] for i in range(query_result["hits"]["total"])]
    return results